In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 24.6 M

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 42,
    max_seq_length = max_seq_length
)

Unsloth 2024.11.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Write an RSpec test for the following Rails method, using described_class as the main test subject.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Project/dataset.json")
dataset = dataset['train']

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset[0]

{'instruction': 'Write an RSpec test for the following Rails method, using described_class as the main test subject.',
 'input': "def show\n        begin\n          referral = ::Referral.find_by_id params['id']\n          raise NotFound.new unless referral.present?\n          patient = referral.survey&.patient\n          access_permission_proc.call(current_user, patient)\n          if referral\n            params[:pass_on] = { referral_payload: true }\n            api_v1_record = Api::V1::Referral.new(params[:id], params, current_user)\n          end\n          respond_to do |format|\n            format.json { render json: api_v1_record.show}\n            format.xml { render :xml => api_v1_record.show.to_xml(:root => :data) }\n          end\n        rescue => e\n          log_error_new_relic(e)\n         render_response(e)\n        end\n      end\n\n      private\n\n      def set_record\n        @api_v1_record = Api::V1::Referral.new(params[:id], params, current_user)\n      end\n\n   

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model, # peft model
    train_dataset = dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=1234,
        output_dir="outputs"
    )
)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30 | Num Epochs = 34
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.125500
2,2.069300
3,3.539100
4,4.181100
5,3.868300
6,2.164500
7,5.197400
8,3.539800
9,2.938400
10,2.567000


In [ ]:
trainer_stats

TrainOutput(global_step=100, training_loss=3.35994677901268, metrics={'train_runtime': 2190.9615, 'train_samples_per_second': 0.365, 'train_steps_per_second': 0.046, 'total_flos': 3.797499307760026e+16, 'train_loss': 3.35994677901268, 'epoch': 28.533333333333335})

In [ ]:
## save the model
model.save_pretrained("./best_model")
tokenizer.save_pretrained('./best_model')

('./best_model/tokenizer_config.json',
 './best_model/special_tokens_map.json',
 './best_model/tokenizer.json')

In [ ]:
## unsloth save model
from unsloth import unsloth_save_model

unsloth_save_model(model, tokenizer, "unsloth_model", )

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... Done.


('unsloth_model', None)

In [ ]:
FastLanguageModel.for_inference(model)

instruction = "Write an RSpec test for the following Rails method, using described_class as the main test subject."
input =  """
def get_report_route_data(action = nil)
  action.present? ? { controller: 'surveys', action: action } : { controller: '', action: '' }
end
"""

# process the input
inputs = tokenizer([alpaca_prompt.format(instruction, input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=5000, temperature=0.1)
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|begin_of_text|>Write an RSpec test for the following Rails method, using described_class as the main test subject.

### Instruction:
Write an RSpec test for the following Rails method, using described_class as the main test subject.

### Input:

def get_report_route_data(action = nil)
  action.present?? { controller:'surveys', action: action } : { controller: '', action: '' }
end


### Response:
```ruby
describe 'get_report_route_data' do
  it'returns a hash with the controller and action' do
    expect(described_class.get_report_route_data).to eq({ controller: '', action: '' })
    expect(described_class.get_report_route_data('show')).to eq({ controller:'surveys', action:'show' })
  end
end
```
<|end_of_text|>


In [ ]:
!pip install flask pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok auth token
!ngrok config add-authtoken 2ocjxMxIGKi4cKChJabmCveRmyt_4akE2WXCv29svPpFxzSKp

app = Flask(__name__)

def generate_rspec(input):
  FastLanguageModel.for_inference(model)
  instruction = "Write an RSpec test for the following Rails method, using described_class as the main test subject."

  # process the input
  inputs = tokenizer([alpaca_prompt.format(instruction, input, "")], return_tensors='pt').to('cuda')
  outputs = model.generate(**inputs, max_new_tokens=3000, temperature=0.1)
  response = tokenizer.batch_decode(outputs)[0]
  return response


@app.route('/', methods=['POST'])
def get_rspec():
    data = request.get_json()
    input = data.get("input")
    print(input)
    result = generate_rspec(input)
    print(result)
    return jsonify({"result": result})

# Start ngrok and Flask app
public_url = ngrok.connect(5000)
print("Server accessible at:", public_url)
app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Server accessible at: NgrokTunnel: "https://00c4-34-145-109-145.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


def parse_response(response)
    if response.blank?
      'Response was empty'
    elsif response['errors'].present?
      response['errors']
    elsif response['id'].blank?
      response['status']
    else
      'Something went wrong. Please try again.'
    end


INFO:werkzeug:127.0.0.1 - - [11/Nov/2024 15:19:17] "POST / HTTP/1.1" 200 -


<|begin_of_text|>Write an RSpec test for the following Rails method, using described_class as the main test subject.

### Instruction:
Write an RSpec test for the following Rails method, using described_class as the main test subject.

### Input:
def parse_response(response)
    if response.blank?
      'Response was empty'
    elsif response['errors'].present?
      response['errors']
    elsif response['id'].blank?
      response['status']
    else
      'Something went wrong. Please try again.'
    end

### Response:
describe 'parse_response' do
  it'returns an error message when the response is blank' do
    response = {}
    expect(described_class.parse_response(response)).to eq('Response was empty')
  end

  it'returns the errors when the response has errors' do
    response = { errors: ['Error 1', 'Error 2'] }
    expect(described_class.parse_response(response)).to eq(['Error 1', 'Error 2'])
  end

  it'returns the status when the response has a status but no id' do
    response